<a href="https://colab.research.google.com/github/alebjanes/fire-susceptibility-mapping/blob/main/Multi_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import load
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation, Input, Concatenate, Conv2DTranspose
from tensorflow.keras import regularizers
import tensorflow as tf
from keras.layers.merge import concatenate
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#DATA
pixel_size = 25
mid_pixel = pixel_size/2 - 0.5

Dataset = np.load('/content/drive/My Drive/MT/Samples/samples' + str(pixel_size) + 'x'+ str(pixel_size) +'_v3.npy')

X = Dataset[:,:,:,1:21]
target = Dataset[:,mid_pixel,mid_pixel,0]
y = np.expand_dims(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X=X_train
y=y_train

del y_test
del X_test
del target
del X_train
del y_train

In [ ]:
#CV
num_folds = 5

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

acc_per_fold_train = []
loss_per_fold_train = []

In [ ]:
def inputlayers(input_shape, batch_normalization, momentum, activation):
  input = Input(shape=input_shape)
  x = Conv2D(32, kernel_size=(3, 3), activation=activation, padding = 'same')(input)
  if batch_normalization: 
    x = BatchNormalization(momentum=0.99)(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Conv2D(64, kernel_size=(3, 3), activation=activation, padding = 'same')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  conv = Conv2D(128, kernel_size=(3, 3), activation=activation, padding = 'same')(x)

  return input, conv

def build_model(patch_size, batch_normalization, momentum, activation, loss, lr):
  #Climatic features
  input1, feature1 = inputlayers((patch_size, patch_size, 6), batch_normalization, momentum = momentum, activation = activation)

  #NDVI feature
  input2, feature2 = inputlayers((patch_size, patch_size, 1), batch_normalization, momentum = momentum, activation = activation)

  #Topograhic features
  input3, feature3 = inputlayers((patch_size, patch_size, 4), batch_normalization, momentum = momentum, activation = activation)

  #Human-related features
  input4, feature4 = inputlayers((patch_size, patch_size, 3), batch_normalization, momentum = momentum , activation = activation)

  #Human-related features
  input5, feature5 = inputlayers((patch_size, patch_size, 6), batch_normalization, momentum = momentum , activation = activation)

  merge = Concatenate(axis = -1)([feature1, feature2, feature3, feature4, feature5])
  
  x = Conv2DTranspose(32, kernel_size=(3,3), dilation_rate=2)(merge)
  x = Conv2D(32, kernel_size=(3, 3), activation=activation, padding = 'same')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Conv2D(64, kernel_size=(3, 3), activation=activation, padding = 'same')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  conv = Conv2D(128, kernel_size=(3, 3), activation=activation, padding = 'same')(x)

  flatten = Flatten()(conv)

  FC2 = Dense(128, activation = activation, kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.01))(flatten)
  dropout2 = Dropout(0.5)(FC2)
  FC3 = Dense(64, activation = activation, kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.01))(dropout2)
  dropout3 = Dropout(0.5)(FC3)
  FC4 = Dense(32, activation = activation, kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.01))(dropout3)
  dropout4 = Dropout(0.5)(FC4)
  if loss == 'binary_crossentropy':
    output = Dense(1, activation = 'sigmoid')(dropout4)
  elif loss == 'sparse_categorical_crossentropy':
    output = Dense(2, activation = 'softmax')(dropout4)  
  
  model = Model(inputs=[input1, input2, input3, input4, input5], outputs = output)
  
  model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.9999, epsilon=1e-07), metrics=['accuracy'])

#kernel_regularizer=regularizers.l1_l2(l1 = 0.001,l2 = 0.01)

  return model

In [ ]:
#Callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=50, min_lr=0.0001, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=100, min_delta=0.01, restore_best_weights=True)

In [ ]:
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(X, y):

  # Define the model architecture
  model = build_model(patch_size = 5, batch_normalization = True, momentum = 0.99, activation = 'relu', loss = 'binary_crossentropy', lr = 0.0001)

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Separate data by group
  climatic = X[train][:,:,:,0:6]
  vegetation = X[train][:,:,:,6:7]
  topographic = X[train][:,:,:,7:11]
  human_related = X[train][:,:,:,11:14]
  LULC = X[train][:,:,:,14:21]

  climatic_val = X[test][:,:,:,0:6]
  vegetation_val = X[test][:,:,:,6:7]
  topographic_val = X[test][:,:,:,7:11]
  human_related_val = X[test][:,:,:,11:14]
  LULC_val = X[test][:,:,:,14:21]

  # Fit data to model
  history = model.fit([climatic, vegetation, topographic, human_related, LULC], y[train],
              batch_size=32,
              epochs=200,
              callbacks=[reduce_lr, early_stop],
              validation_data = ([climatic_val, vegetation_val, topographic_val, human_related_val, LULC_val], y[test]), verbose = 0)
  
  # Generate generalization metrics
  scores = model.evaluate([climatic_val, vegetation_val, topographic_val, human_related_val, LULC_val], y[test], verbose=0)
  print(f'Score for fold {fold_no} on validation: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = model.evaluate([climatic, vegetation, topographic, human_related, LULC], y[train], verbose=0)
  print(f'Score for fold {fold_no} on training: {model.metrics_names[0]} of {scores_train[0]}; {model.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train.append(scores_train[1] * 100)
  loss_per_fold_train.append(scores_train[0])

  # Increase fold number
  fold_no = fold_no + 1


  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Dataset size:', X.shape[0])
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('Score per fold on training set')
for i in range(0, len(acc_per_fold_train)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_train[i]} - Accuracy: {acc_per_fold_train[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_train)} (+- {np.std(acc_per_fold_train)})')
print(f'> Loss: {np.mean(loss_per_fold_train)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Restoring model weights from the end of the best epoch.
Epoch 00106: early stopping
Score for fold 1 on validation: loss of 0.6672959923744202; accuracy of 62.506675720214844%
Score for fold 1 on training: loss of 0.6526517271995544; accuracy of 64.34770822525024%
------------------------------------------------------------------------
Training for fold 2 ...
Restoring model weights from the end of the best epoch.
Epoch 00108: early stopping
Score for fold 2 on validation: loss of 0.6667557954788208; accuracy of 62.23962903022766%
Score for fold 2 on training: loss of 0.642261803150177; accuracy of 65.05096554756165%
------------------------------------------------------------------------
Training for fold 3 ...
Restoring model weights from the end of the best epoch.
Epoch 00105: early stopping
Score for fold 3 on validation: loss of 0.670974612236023; accuracy of 62.880539894104004%
Score 

In [ ]:
history = model1.fit([climatic, vegetation, topographic, human_related, LULC], y_train, epochs=200, verbose=1, batch_size = 32,
                    validation_data=([climatic_val, vegetation_val, topographic_val, human_related_val, LULC_val], y_test), callbacks = [early_stop, reduce_lr])

In [ ]:
model1.save("/content/drive/My Drive/MT/checkpoints/Multi-input/Multi-input25")